In [1]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import argparse
import sys
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
np.set_printoptions(edgeitems=3,infstr='inf',linewidth=75, nanstr='nan', precision=8,suppress=False, threshold=1000, formatter=None)
np.random.seed(2)
import warnings
warnings.filterwarnings('ignore')
print ("cargo")

/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


cargo


In [2]:
def datos():
    
    print("Comenzo la carga...")
    train_req = pd.read_excel("train_requerimientos.xlsx")
    train_cliente = pd.read_excel("train_clientes.xlsx")
    test_cliente = pd.read_excel('test_clientes.xlsx')
    test_req = pd.read_excel('test_requerimientos.xlsx')
    #'CODMES'

    
    #Y_train = train_cliente['ATTRITION']
    
    group_train2 =  train_req.groupby(["ID_CORRELATIVO","TIPO_REQUERIMIENTO2","DICTAMEN"])
    group_test2 =  test_req.groupby(["ID_CORRELATIVO","TIPO_REQUERIMIENTO2","DICTAMEN"])

    df_train_req=group_train2.size().to_frame('size').reset_index()
    df_test_req=group_test2.size().to_frame('size').reset_index()

    df_train_req['FUSION']=df_train_req['TIPO_REQUERIMIENTO2']+" "+df_train_req['DICTAMEN']
    df_test_req['FUSION']=df_train_req['TIPO_REQUERIMIENTO2']+" "+df_train_req['DICTAMEN']

    pivot=df_train_req.pivot_table(values='size',index="ID_CORRELATIVO", columns=["TIPO_REQUERIMIENTO2","FUSION"],aggfunc=np.sum)
    pivot_test=df_test_req.pivot_table(values='size',index="ID_CORRELATIVO", columns=["TIPO_REQUERIMIENTO2","FUSION"],aggfunc=np.sum)

    pivot.fillna(value=0, inplace=True)
    pivot_test.fillna(value=0, inplace=True)

    pivot_total_reclamo= pivot["Reclamo"].sum(axis=1)
    pivot_total_reclamo_test= pivot_test["Reclamo"].sum(axis=1)
    #pivot_total_reclamo=pivot_total_reclamo.to_frame('SUMA RECLA')
    pivot_total_solicitud= pivot["Solicitud"].sum(axis=1)
    pivot_total_solicitud_test= pivot_test["Solicitud"].sum(axis=1)
    #pivot_total_solicitud=pivot_total_solicitud.to_frame('SUMA SOLI')
    #pivot.reset_index(inplace=True)
    #pivot_total_reclamo.reset_index(inplace=True)
    #pivot_total_reclamo

    pivot_no_procede_reclamo=pivot["Reclamo"]["Reclamo NO PROCEDE"]
    pivot_reclamo_parcial=pivot["Reclamo"]["Reclamo PROCEDE PARCIAL"]

    pivot_no_procede_reclamo_test=pivot_test["Reclamo"]["Reclamo NO PROCEDE"]
    pivot_reclamo_parcial_test=pivot_test["Reclamo"]["Reclamo PROCEDE PARCIAL"]
    #pivot_no_procede_reclamo.to_frame('No Reclamo')
    pivot_no_procede_solicitud=pivot["Solicitud"]["Solicitud NO PROCEDE"]
    pivot_solicitud_parcial=pivot["Solicitud"]["Solicitud PROCEDE PARCIAL"]

    pivot_no_procede_solicitud_test=pivot_test["Solicitud"]["Solicitud NO PROCEDE"]
    pivot_solicitud_parcial_test=pivot_test["Solicitud"]["Solicitud PROCEDE PARCIAL"]


    #pivot_no_procede_reclamo.to_frame('No Solicitud')

    requeremiento={}
    requeremiento_test={}
    requerimiento={"reclamos":pivot_total_reclamo, "no_reclamos":pivot_no_procede_reclamo,"reclamo_parcial":pivot_reclamo_parcial ,"solicitudes":pivot_total_solicitud,"no_solicitudes":pivot_no_procede_solicitud,"solicitudes_parcial":pivot_solicitud_parcial}
    requerimiento_test={"reclamos":pivot_total_reclamo_test, "no_reclamos":pivot_no_procede_reclamo_test ,"reclamo_parcial":pivot_reclamo_parcial_test,"solicitudes":pivot_total_solicitud_test,"no_solicitudes":pivot_no_procede_solicitud_test,"solicitudes_parcial":pivot_solicitud_parcial_test}


    df_requerimiento=pd.DataFrame(requerimiento)
    df_requerimiento_test=pd.DataFrame(requerimiento_test)

    df_requerimiento["no_reclamos"] = np.where(df_requerimiento["reclamos"]==0, 0, df_requerimiento["no_reclamos"]/df_requerimiento["reclamos"])
    df_requerimiento["reclamo_parcial"] = np.where(df_requerimiento["reclamos"]==0, 0, df_requerimiento["reclamo_parcial"]/df_requerimiento["reclamos"])

    df_requerimiento_test["no_reclamos"] = np.where(df_requerimiento_test["reclamos"]==0, 0, df_requerimiento_test["no_reclamos"]/df_requerimiento_test["reclamos"])
    df_requerimiento_test["reclamo_parcial"] = np.where(df_requerimiento_test["reclamos"]==0, 0, df_requerimiento_test["reclamo_parcial"]/df_requerimiento_test["reclamos"])

    #procesado=pd.DataFrame(pivot_total_reclamo,pivot_total_solicitud)
    df_requerimiento["no_solicitudes"] = np.where(df_requerimiento["solicitudes"]==0, 0, df_requerimiento["no_solicitudes"]/df_requerimiento["solicitudes"])
    df_requerimiento["solicitudes_parcial"] = np.where(df_requerimiento["solicitudes"]==0, 0, df_requerimiento["solicitudes_parcial"]/df_requerimiento["solicitudes"])

    df_requerimiento_test["no_solicitudes"] = np.where(df_requerimiento_test["solicitudes"]==0, 0, df_requerimiento_test["no_solicitudes"]/df_requerimiento_test["solicitudes"])
    df_requerimiento_test["solicitudes_parcial"] = np.where(df_requerimiento_test["solicitudes"]==0, 0, df_requerimiento_test["solicitudes_parcial"]/df_requerimiento_test["solicitudes"])

    df_requerimiento.reset_index(inplace=True)
    df_requerimiento_test.reset_index(inplace=True)

    train=pd.merge(train_cliente,df_requerimiento, on="ID_CORRELATIVO", how='left')
    test=pd.merge(test_cliente,df_requerimiento_test, on="ID_CORRELATIVO", how='left')
    
    values = {'no_reclamos': 0, 'no_solicitudes': 0, 'reclamos': 0,'solicitudes': 0 ,'no_reclamos': 0,'no_solicitudes': 0,'reclamo_parcial':0,'solicitudes_parcial':0}
    train.fillna(value=values, inplace=True)
    test.fillna(value=values, inplace=True)
    print ("antes")
    print (train.shape)
    ##ATTRITION
    train.drop(['ID_CORRELATIVO','CODMES'], axis=1, inplace=True)
    test.drop(['ID_CORRELATIVO','CODMES'], axis=1, inplace=True)
    print ("despues")
    print (train.shape)

    train.dropna(inplace=True)
    test.dropna(inplace=True)
    
    train= pd.get_dummies(train)
    
    #X_train, Y_train, X_test, Y_test
    X_train=train.drop(['ATTRITION'], axis=1)
    
    Y_train_c=train[['ATTRITION']]
    Y_train = Y_train_c.copy()
    Y_train['NATTRITION']=1-Y_train['ATTRITION']   
    
    X_train_p, X_test_p, Y_train_p,Y_test_p = train_test_split(X_train,Y_train,train_size=0.85, random_state=0)
    
    X_train_p = X_train_p.as_matrix()
    Y_train_p = Y_train_p.as_matrix()
    X_test_p = X_test_p.as_matrix()
    Y_test_p = Y_test_p.as_matrix()
    ##normalizar datos de entrada
    scaler= MinMaxScaler()
    scaler.fit(X_train_p)
    
    X_train_p = scaler.transform(X_train_p)
    X_test_p = scaler.transform(X_test_p)
    
    print ("Termino cargar")
    return X_train_p.T, Y_train_p.T, X_test_p.T, Y_test_p.T

In [3]:
def create_placeholders(n_x, n_y):
    ### START CODE HERE ### (approx. 2 lines)
    X = tf.placeholder(tf.float32,shape=(n_x, None),name="X")
    Y = tf.placeholder(tf.float32,shape=(n_y, None),name="Y")
    ### Otras variables ###
    
     # variable learning rate
    lr = tf.placeholder(tf.float32)
    # train/test selector for batch normalisation
    tst = tf.placeholder(tf.bool)
    # training iteration
    iter = tf.placeholder(tf.int32)   
    # training iteration
    pkeep = tf.placeholder(tf.float32)      
    return X, Y, tst, iter,pkeep

def initialize_parameters(layer,tipo=1):
    parameters={}
    tf.set_random_seed(1)                   # so that your "random" numbers match ours
    L=len(layer)
    for i in range(1, L):
        if(tipo==1):
            parameters['W' + str(i)] =tf.get_variable('W' + str(i), [layer[i], layer[i-1]], initializer = tf.contrib.layers.xavier_initializer(seed = 1))        
            parameters['b' + str(i)] =tf.get_variable('b' + str(i), [layer[i], 1], initializer = tf.zeros_initializer())
        
        if(tipo==2):
            parameters['W' + str(i)] =tf.Variable(np.random.randn(layer[i], layer[i-1])*0.01,dtype=tf.float32,name='W' + str(i))        
            parameters['b' + str(i)] =tf.get_variable('b' + str(i), [layer[i], 1], initializer = tf.zeros_initializer())
            
    
    return parameters

def batchnorm(Ylogits, is_test,iter):
    Ylogits = tf.transpose(Ylogits)
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999,iter) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    
    mean, variance = tf.nn.moments(Ylogits, [0])
    
    update_moving_averages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, None, None, bnepsilon)
    Ybn = tf.transpose(Ybn)
    return Ybn, update_moving_averages

def forward_propagation(X, parameters,tst,iter,pkeep):   
    maximo= (len(parameters))
    cont =1
    l_update_ema=[]
    for i in range(1,maximo+1,2):
        if (cont==((maximo)/2)):
            break
        if(i==1):
            Z=linear_forward(X,parameters['W'+str(cont)],parameters['b'+str(cont)])
            #Z = tf.matmul(parameters['W'+str(cont)],X)
            ZB, update_ema = batchnorm(Z, tst,iter)
            A = tf.nn.relu(ZB)                                              
            #Ad = sess.run(tf.nn.dropout(A, pkeep))
            Ad = tf.nn.dropout(A, pkeep)
        else:
            #Z = tf.matmul(parameters['W'+str(cont)],Ad)
            Z=linear_forward(Ad,parameters['W'+str(cont)],parameters['b'+str(cont)])
            ZB, update_ema = batchnorm(Z, tst,iter)
            A = tf.nn.relu(ZB)                                        
            #Ad = tf.nn.dropout(A, pkeep)
            Ad = tf.nn.dropout(A, pkeep)
        l_update_ema.append(update_ema)
        cont=cont+1
            
    #Z3 = tf.add(tf.matmul(parameters['W'+str(cont)],Ad),parameters['b'+str(cont)])    
    Z3=linear_forward(Ad,parameters['W'+str(cont)],parameters['b'+str(cont)])
    ##softmax
    Y_Y = tf.nn.softmax(tf.transpose(Z3))
    if((cont==3)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1])
    if((cont==4)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2])
    if((cont==5)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],l_update_ema[3])  
    if((cont==6)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],l_update_ema[3],l_update_ema[4])
    if((cont==7)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],l_update_ema[3],l_update_ema[4],l_update_ema[5])
    if((cont==8)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],l_update_ema[3],l_update_ema[4],l_update_ema[5],l_update_ema[6])
    if((cont==9)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],l_update_ema[3],l_update_ema[4],l_update_ema[5],l_update_ema[6],l_update_ema[7])
    if((cont==10)):
        update_ema = tf.group(l_update_ema[0],l_update_ema[1],l_update_ema[2],l_update_ema[3],l_update_ema[4],l_update_ema[5],l_update_ema[6],l_update_ema[7],l_update_ema[8])        
    return Z3,update_ema,Y_Y

def linear_forward(A, W, b):
    Z = tf.add(tf.matmul(W,A),b)  
    return Z

# GRADED FUNCTION: compute_cost 
def compute_cost(Z3, Y):
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    ### END CODE HERE ###
    
    return cost

def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

def model(X_train, Y_train, X_test, Y_test, 
          learning_rate = 0.0001,
          num_epochs = 30, 
          minibatch_size = 32, 
          print_cost = False,
          pkeepv=1.0,
         layer=[],
         inicializa=1):

    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    cost_train = []                                        # To keep track of the cost
    cost_test = []
    
    #print (n_x)
    #print (m)
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X, Y,tst,iter,pkeep= create_placeholders(n_x, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    ### inicializar los parametros y la arquitectura de la red
    #parameters = initialize_parameters(n_x,n_y,2)
    parameters = initialize_parameters(layer,1)
    #print (parameters)
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3,update_ema,Y_Y = forward_propagation(X, parameters,tst,iter,pkeep)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z3,Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()
    
    ##### Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        i=0
        for epoch in range(num_epochs):
            i=i+1
            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            #print ("mini "+str(num_minibatches))
            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                #_ , minibatch_cost, _ = sess.run([optimizer, cost, update_ema], feed_dict={X: minibatch_X, Y: minibatch_Y, tst: False, iter: i,pkeep: pkeepv})
                ### END CODE HERE ###
                minibatch_cost = sess.run(cost, feed_dict={X: minibatch_X, Y: minibatch_Y, tst: False,pkeep:pkeepv})
                sess.run(optimizer, feed_dict={X: minibatch_X, Y: minibatch_Y, tst: False, iter: i, pkeep:pkeepv})
                sess.run(update_ema, feed_dict={X: minibatch_X, Y: minibatch_Y, tst: False, iter: i, pkeep:pkeepv})
                
                epoch_cost += minibatch_cost / num_minibatches
                #if(i==30):
                #    break

            # Print the cost every epoch
            if print_cost == True and epoch % 10 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            #if print_cost == True and epoch % 5 == 0:
            #    cost_train.append(epoch_cost)
            #    c_test = sess.run([ cost], feed_dict={X: minibatch_X, Y: minibatch_Y, tst: True,pkeep: 1.0})
            #    cost_test.append(c_test)
        '''
        if(print_cost== True):
            # plot the cost
            plt.figure("Figura1")
            plt.plot(np.squeeze(cost_train))
            plt.ylabel('cost')
            plt.xlabel('iterations (per tens)')
            plt.title("Train Learning rate =" + str(learning_rate))

            plt.figure("Figura2")
            plt.plot(np.squeeze(cost_test))
            plt.ylabel('cost')
            plt.xlabel('iterations (per tens)')
            plt.title("Test Learning rate =" + str(learning_rate))        


            plt.figure("Figura3")
            plt.plot(np.squeeze(cost_train))
            plt.plot(np.squeeze(cost_test))
            plt.ylabel('cost')
            plt.xlabel('iterations (per tens)')
            plt.title("Test Learning rate =" + str(learning_rate)) 
            plt.legend(loc=4)
            plt.show()      
          '''

        
        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        #print ("Parameters have been trained!")

        # Calculate the correct predictions, xq no recibe a3. Claro no lo tengo. ¿como sabe que debe calcularlo?
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))
        #z3_value = z3
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        acc_train=sess.run(accuracy, feed_dict ={X: X_train, Y: Y_train,tst: False,pkeep: 1.0})
        acc_test, A3_value=sess.run([accuracy,Y_Y] ,feed_dict ={X: X_test, Y: Y_test,tst: True,pkeep: 1.0})
        
        #print ("Train Accuracy :", accuracy.eval({X: X_train, Y: Y_train,tst: False,pkeep: 1.0}))
        #print ("Test Accuracy: ", accuracy.eval({X: X_test, Y: Y_test,tst: True,pkeep: 1.0}))
        '''
        print ("Train Accuracy:", sess.run(accuracy, feed_dict ={X: X_train, Y: Y_train})

        print ("Test Accuracy:", sess.run(accuracy, feed_dict ={X: X_test, Y: Y_test})
        
        '''
        
        #print ("-----Fin prueba------")
        return parameters,acc_train,acc_test,A3_value
print ("Cargo Modelo")

Cargo Modelo


In [ ]:
%%time
X_train, Y_train, X_test, Y_test = datos()
print (X_train.shape)
#Y_train=Y_train.reshape(Y_train.shape[0],1)
print (Y_train.shape)
print (X_test.shape)
#Y_test=Y_test.reshape(Y_test.shape[0],1)
print (Y_test.shape)

Comenzo la carga...
antes
(70000, 59)
despues
(70000, 57)
Termino cargar
(84, 47822)
(2, 47822)
(84, 8440)
(2, 8440)
CPU times: user 1min 22s, sys: 532 ms, total: 1min 22s
Wall time: 7min 33s


In [ ]:
%%time
np.random.seed(2)
#a_learning_rate = [0.0001, 0.009, 0.005, 0.001, 0.09, 0.05, 0.01, 0.9, 0.5, 0.1]
#a_pkeepv = [1,0.95,0.90,0.85,0.80]
(n_x, m) = X_train.shape                         
n_y = Y_train.shape[0] 
l_layer=[]
#l_layer.append([n_x,25,12,n_y])
#l_layer.append([n_x,100,50,25,n_y])
#l_layer.append([n_x,200,100,50,25,n_y])
#l_layer.append([n_x,1000,500,n_y])
#l_layer.append([n_x,1000,200,25,n_y])
l_layer.append([n_x,400,300,200,100,50,25,n_y])
#l_layer.append([n_x,800,400,200,100,50,25,n_y])
#l_layer.append([n_x,1600,800,400,200,100,50,25,n_y])
### E_learning_rate

l_el = np.random.uniform(-4,-2,10)
a_learning_rate=np.unique(np.around(np.power(10,l_el),decimals=5))
#l_el=-4*np.random.rand(10)

#a_learning_rate=np.unique(np.around(np.power(10,l_el),decimals=3))
##a_learning_rate = [0.0001]
inicializa=1
#a_pkeepv = [1,0.85],no generalize al inicio
a_pkeepv = [1]
nume=1000
result=[]
for e_layer in l_layer:
    for e_learning_rate in a_learning_rate:
        for e_pkeepv in a_pkeepv:
            print ("Arquitectura: ","-".join(map(str, e_layer))," Learning Rate: ", e_learning_rate, " Dropout ", e_pkeepv," Epoch: ", nume," Inicializar: ", inicializa)
            parameters,acc_train,acc_test,A3_value = model(X_train, 
                                                  Y_train, 
                                                  X_test, 
                                                  Y_test,
                                                  learning_rate=e_learning_rate,
                                                  pkeepv=e_pkeepv,
                                                  num_epochs=nume,
                                                  print_cost=True,
                                                  layer=e_layer,
                                                inicializa=2)
            result.append([e_layer,e_learning_rate,e_pkeepv,acc_train,acc_test,nume,inicializa])
            print ("Accuracity Train: ", acc_train,"Accuracity Test: " ,acc_test)
            print("----- ----- -----")

fecha=time.strftime("%d-%m-%Y-%H-%M-%S")
aresu=np.asarray(result)
aresu=aresu[np.argsort(aresu[:, 4])].tolist()
resultado = '\n'.join(map(str, aresu))
f = open ('resultado/archivo'+fecha+'.txt',"w")
f.write(resultado)
f.close()
print("----Termino-----")

Arquitectura:  84-400-300-200-100-50-25-2  Learning Rate:  0.00011  Dropout  1  Epoch:  1000  Inicializar:  1
Cost after epoch 0: 0.516938
Cost after epoch 10: 0.275040
Cost after epoch 20: 0.244974
Cost after epoch 30: 0.223764
Cost after epoch 40: 0.206544
Cost after epoch 50: 0.193939
Cost after epoch 60: 0.185581
Cost after epoch 70: 0.175643
Cost after epoch 100: 0.155531
Cost after epoch 110: 0.150398
Cost after epoch 120: 0.148091
Cost after epoch 130: 0.143660
Cost after epoch 140: 0.138519
Cost after epoch 150: 0.134246
Cost after epoch 160: 0.131731
Cost after epoch 170: 0.127130
Cost after epoch 180: 0.125983
Cost after epoch 190: 0.123189
Cost after epoch 200: 0.120909
Cost after epoch 210: 0.117970
Cost after epoch 220: 0.114958
Cost after epoch 230: 0.113067
Cost after epoch 240: 0.112358
Cost after epoch 250: 0.110203
Cost after epoch 260: 0.107040
Cost after epoch 270: 0.106604
Cost after epoch 280: 0.104753
Cost after epoch 290: 0.101441
Cost after epoch 300: 0.100439


KeyboardInterrupt: 

In [17]:
print (A3_value[0:10,0:2])

[[ 0.02828412  0.97171581]
 [ 0.04219292  0.95780712]
 [ 0.04681063  0.95318937]
 [ 0.05038058  0.94961935]
 [ 0.39675713  0.60324293]
 [ 0.1171367   0.88286334]
 [ 0.09787513  0.90212482]
 [ 0.06176955  0.93823045]
 [ 0.15814407  0.84185594]
 [ 0.06751215  0.93248779]]


In [10]:
resultado = '\n'.join(map(str, result))

In [65]:
A3_value.shape

(10000, 10)

In [27]:
aresu=np.asarray(result)
aresu

array([[list([84, 25, 12, 2]), 0.021473952157175989, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.001314322752836385, 0.85, 0.88593113,
        0.88720381],
       [list([84, 25, 12, 2]), 0.99894712366592076, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.061754656372527347, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.25880724917679454, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.42721414228233884, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.17987017387999535, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.041472201678756003, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.025877963923366044, 0.85, 0.87800592,
        0.88364929],
       [list([84, 25, 12, 2]), 0.0069941197526672593, 0.85, 0.88197899,
        0.88412321],
       [list([84, 100, 50, 25, 2]), 0.021473952157175989, 0.85, 0.88513654,
       

In [ ]:
[list([84, 25, 12, 2]), 0.0069941197526672593, 0.85, 0.88197899,
        0.88412321],
   [list([84, 25, 12, 2]), 0.0069941197526672593, 0.85, 0.88197899,
        0.88412321]
   [list([84, 100, 50, 25, 2]), 0.0069941197526672593, 0.85,
        0.88718581, 0.88827014]], dtype=object)

In [38]:
aresu=np.asarray(result)
aresu=aresu[np.argsort(aresu[:, 4])].tolist()
resultado = '\n'.join(map(str, aresu))
f = open ('resultado/archivoprueba'+fecha+'.txt',"w")
f.write(resultado)
f.close()


[[[84, 25, 12, 2], 0.021473952157175989, 0.85, 0.87800592, 0.88364929],
 [[84, 100, 50, 25, 2], 0.17987017387999535, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.99894712366592076, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.061754656372527347, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.25880724917679454, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.42721414228233884, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.17987017387999535, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.041472201678756003, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.025877963923366044, 0.85, 0.87800592, 0.88364929],
 [[84, 100, 50, 25, 2], 0.42721414228233884, 0.85, 0.87800592, 0.88364929],
 [[84, 100, 50, 25, 2], 0.25880724917679454, 0.85, 0.87800592, 0.88364929],
 [[84, 100, 50, 25, 2], 0.061754656372527347, 0.85, 0.87819415, 0.88364929],
 [[84, 100, 50, 25, 2], 0.99894712366592076, 0.85, 0.87800592, 0.88364929],
 [[84, 25, 12, 2], 0.0069941197526672593, 0.85,